In [1]:
!pip show chromadb

Name: chromadb
Version: 0.6.3
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: D:\doc-qa-app\new_env\Lib\site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, rich, tenacity, tokenizers, tqdm, typer, typing_extensions, uvicorn
Required-by: 


In [2]:
import os
os.environ["AI21_API_KEY"] = "kV9b894VQ7EVSAj1CYS76V0yEZ1dmxjY"

In [3]:
from langchain_ai21 import ChatAI21
from langchain_ai21 import AI21Embeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader

In [4]:
from langchain.document_loaders import PyPDFLoader

In [5]:
# Step 2: Load the PDF document
pdf_path = "54.SFSP - Policy Wording.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

ImportError: pypdf package not found, please install it with `pip install pypdf`

In [5]:
documents[0]

Document(metadata={'source': 'data/54.SFSP - Policy Wording.pdf', 'page': 0}, page_content=' \n                      Liberty Videocon General Insurance  Company Limited – Policy W ording - SFSP                        1  \nSTANDARD FIRE AND SPECIAL PERILS POLICY  \n(MATERIAL DAMAGE)  \n \n \nIN CONSIDERATION OF the Insured named in the Schedule hereto having paid to the \nLiberty Videocon  General Insurance Company Limited  (hereinafter called the \nCompany) the full premium mentioned in t he said schedule, THE COMPANY AGREES, \n(Subject to the Conditions and Exclusions contained herein or endorsed or otherwise \nexpressed hereon) that if after payment of the premium the Property insured described in \nthe said Schedule or any part of such Propert y be destroyed or damaged by any of the perils \nspecified hereunder during the period of insurance named in the said schedule or of any \nsubsequent period in respect of which the Insured shall have paid and the Company shall \nhave accepted 

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100)
# text = text_splitter.split_documents(documents)

# Split the documents into chunks that are under 2000 characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1900, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)

In [7]:
split_documents[10].page_content

'thereof to the Company and shall within 15 days after the loss or damage, or \nsuch further time as the Company may in writing allow in that behalf, deliver to \nthe Company  \n  \na)  A claim in writing for the loss or d amage containing as particular an account as \nmay be reasonably practicable of all the several articles or items or property \ndamaged or destroyed, and of the amount of the loss or damage thereto \nrespectively, having regard to their value at the time of the loss or damage not \nincluding profit of any kind.  \n \n   b)  Particulars of all other insurances, if any  \n \nThe Insured shall also at all times at his own expense produce, procure and give to \nthe Company all such further particulars, plans, specification books , vouchers, \ninvoices, duplicates or copies thereof, documents, investigation reports \n(internal/external), proofs and information with respect to the claim and the origin \nand cause of the loss and the circumstances under which the loss or da

In [8]:
len(split_documents)

17

In [9]:
embedding = AI21Embeddings()

In [10]:


# Set up AI21 embeddings
embedding = AI21Embeddings()

# Directory to persist the Chroma vector store
persist_directory = 'db'

# Batch processing parameters
batch_size = 50  # Adjust based on your system's capabilities

# Process and store documents in batches
for i in range(0, len(split_documents), batch_size):
    batch = split_documents[i:i+batch_size]
    
    # Ensure each chunk is under 2000 characters
    valid_batch = [doc for doc in batch if len(doc.page_content) <= 2000]
    
    if i == 0:
        vectordb = Chroma.from_documents(documents=valid_batch, embedding=embedding, persist_directory=persist_directory)
    else:
        vectordb.add_documents(documents=valid_batch)
    
    print(f"Processed batch {i//batch_size + 1} of {len(split_documents)//batch_size + 1}")

# Persist the vector store to disk
vectordb.persist()
print("All documents processed and saved.")



Processed batch 1 of 1
All documents processed and saved.


c:\Users\Harisankar\anaconda3\envs\genaienv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [6]:
from langchain.prompts import PromptTemplate
from ai21 import AI21Client
from ai21.models.chat import ChatMessage

In [7]:
client = AI21Client(api_key="kV9b894VQ7EVSAj1CYS76V0yEZ1dmxjY")


In [8]:
# Define the improved prompt template
PROMPT_TEMPLATE = """
You are an AI assistant specialized in explaining insurance policies in simple terms. Your task is to answer the user's question based on the following context from an insurance policy:

{context}

Question: {question}

Please provide a clear, concise, and easy-to-understand answer. Follow these guidelines:
1. Identify the key conditions or points relevant to the question.
2. Explain each point briefly, avoiding insurance jargon where possible.
3. If a term is complex, provide a simple explanation in parentheses.
4. Use bullet points for clarity.
5. If the context doesn't fully answer the question, mention this and provide the most relevant information available.
6. Limit your response to the most important points to ensure clarity and brevity.

Remember, your goal is to help a non-expert understand the key points of the policy.
"""

In [9]:
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context", "question"])


In [77]:
def get_relevant_context(question, k=5):
    print(f"Retrieving context for question: {question}")
    results = vectordb.similarity_search(question, k=k)
    context = "\n".join([doc.page_content for doc in results])
    print(f"Retrieved context (first 200 chars): {context[:200]}...")
    return context

In [75]:
# def get_relevant_context(question, k=5, similarity_threshold=0.3):
#     print(f"Retrieving context for question: {question}")
#     results = vectordb.similarity_search_with_score(question, k=k)
    
#     relevant_docs = [
#         (doc.page_content, score) for doc, score in results 
#         if score >= similarity_threshold
#     ]
    
#     if not relevant_docs:
#         print("No relevant documents found.")
#         return "No relevant documents found"
    
#     context = "\n".join([doc for doc, _ in relevant_docs])
#     print(f"Retrieved context (first 200 chars): {context[:200]}...")
#     return context

In [78]:
get_relevant_context("Fire Insurance")

Retrieving context for question: Fire Insurance
Retrieved context (first 200 chars): Liberty Videocon General Insurance  Company Limited – Policy W ording - SFSP                        3 given as an “add on cover” the words “excluding those resulting from earthquake , 
volcanic erupti...


'Liberty Videocon General Insurance  Company Limited – Policy W ording - SFSP                        3 given as an “add on cover” the words “excluding those resulting from earthquake , \nvolcanic eruption or other convulsions of nature” shall stand deleted. ) \n  \nVII   Impact Damage  \n \n Loss of or visible physical damage or destruction  caused to the property insured  due \nto impact by any Rail/ Road vehicle or animal by direct contact not belonging    to or \nowned by  \n   \na) the Insured or any occupier of the premises or  \n  \nb) their employees while act ing in the course of their employment.  \n \nVIII  Subsidence and Landslide including Rock slide  \n \nLoss, destruction or damage directly caused by Subsidence of part of the site on  \nwhich the property stands or Land slide/Rock slide excluding:  \n  \n         a)    the normal cracking, settlement or bedding down of new structures  \n \n         b)    the settlement or movement of made up ground  \n \n         c)    co

In [79]:
import requests
import time

In [80]:
def answer_question(question):
    try:
        start_time = time.time()
        context = get_relevant_context(question)
        full_prompt = prompt.format(context=context, question=question)
        
        print("Sending request to AI21 API...")
        messages = [
            ChatMessage(role="system", content="You are an AI assistant specialized in insurance policies. Answer questions as best as you can based only on the provided context."),
            ChatMessage(role="user", content=full_prompt)
        ]
        
        response = client.chat.completions.create(
            model="jamba-1.5-mini",
            messages=messages,
            max_tokens=500,
            timeout=30  # Set a 30-second timeout
        )
        
        end_time = time.time()
        print(f"Request completed in {end_time - start_time:.2f} seconds")
        
        return response.choices[0].message.content
    except requests.exceptions.Timeout:
        return "The request to AI21 timed out. Please try again later."
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [81]:
# Example usage
print(answer_question("What are the different Fire insurance conditions in the policy wordings? Make it into points"))

Retrieving context for question: What are the different Fire insurance conditions in the policy wordings? Make it into points
Retrieved context (first 200 chars): 5.  Loss, destruction or damage to bullion or unset precious stones, any curios or   
          works of art for an amount exceeding Rs. 10000/ -, goods held  in trust or on  
          commission, ma...
Sending request to AI21 API...
Request completed in 4.96 seconds
Here are the key conditions related to Fire insurance in the policy wordings:

* **Exclusions for Fire Damage:**


  + Fire caused by the property's own fermentation, natural heating, or spontaneous combustion.
  + Fire resulting from any heating or drying process.
  + Fire caused by burning of insured property by order of any Public Authority.
	
* **Exclusions for Other Perils:**


  + Damage to boilers, economizers, or other vessels/machinery/apparatus (in which steam is generated) or their contents due to their own explosion/implosion.
  + Damage caused by ce

In [82]:
from datetime import datetime

In [84]:
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'08_23_2024_15_53_15'

In [10]:
from ai21 import AI21Client
from ai21.models.chat import ChatMessage
import time
import requests
from dotenv import load_dotenv
import os


In [11]:

# Load environment variables
load_dotenv()

def simple_ai21_chat(question):
    try:
        # Initialize AI21 client
        api_key = os.getenv("AI21_API_KEY")
        if not api_key:
            raise ValueError("AI21_API_KEY environment variable is not set.")
        
        client = AI21Client(api_key=api_key)
        
        start_time = time.time()
        print("Sending request to AI21 API...")
        
        # Create messages for the chat
        messages = [
            ChatMessage(
                role="system", 
                content="You are a helpful AI assistant. Answer questions clearly and concisely."
            ),
            ChatMessage(
                role="user", 
                content=question
            )
        ]
        
        # Make the API call
        response = client.chat.completions.create(
            model="jamba-1.5-mini",
            messages=messages,
            max_tokens=500,
            temperature=0.7,
            timeout=30
        )
        
        end_time = time.time()
        print(f"Request completed in {end_time - start_time:.2f} seconds")
        
        return response.choices[0].message.content
        
    except requests.exceptions.Timeout:
        return "The request to AI21 timed out. Please try again later."
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [12]:

# Test the function
if __name__ == "__main__":
    while True:
        question = input("\nEnter your question (or 'quit' to exit): ")
        if question.lower() == 'quit':
            break
            
        print("\nGetting response...")
        response = simple_ai21_chat(question)
        print("\nResponse:", response)


Getting response...
Sending request to AI21 API...
Request completed in 1.53 seconds

Response: My role is to assist you by providing helpful and professional responses to your questions. Whether it's solving problems, offering explanations, or helping with tasks, I'm here to help. If you have any specific questions or need assistance with something, feel free to ask!

Getting response...
Sending request to AI21 API...
Request completed in 2.72 seconds

Response: Good sleep is crucial for overall health and well-being. It impacts physical health, mental health, and cognitive function. Here are some key benefits:

1. **Physical Health**:


  * **Immune Function**: Adequate sleep strengthens the immune system, helping the body fight off illnesses.
  * **Heart Health**: It reduces the risk of heart disease, high blood pressure, and stroke.
  * **Weight Management**: Good sleep helps regulate hormones that control appetite, reducing the risk of obesity.
	
2. **Mental Health**:


  * **Moo

[2025-01-30 19:07:56 - ai21 - ERROR] Calling POST https://api.ai21.com/studio/v1/chat/completions failed with a non-200 response code: 422



Response: An error occurred: Failed with http status code: 422 (UnprocessableEntity). Details: {"detail":{"error":["Value error, message content must not be an empty string"]}}

Getting response...
Sending request to AI21 API...


[2025-01-30 19:08:04 - ai21 - ERROR] Calling POST https://api.ai21.com/studio/v1/chat/completions failed with a non-200 response code: 422



Response: An error occurred: Failed with http status code: 422 (UnprocessableEntity). Details: {"detail":{"error":["Value error, message content must not be an empty string"]}}
